<a href="https://colab.research.google.com/github/faisalnawazmir/Econometrics-ML_for_Finance/blob/main/Copy_of_ayesha__Fama_MacBeth_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Real example

In [43]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm 
from tabulate import tabulate
import time
np.random.seed(12345)

In [44]:
#copying data from excel file 25 portfoli
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
file = r'/gdrive/My Drive/data/Factors FF_ayesha.xlsx'
df = pd.read_excel(file)
df.head()

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


,Date,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p20,p21,p22,p23,p24,p25,Mkt_rf,SMB,HML,WML,CMA,RMW,Reg
0,2000-01-01,0.178104,0.149602,0.142498,0.121144,0.110616,0.117735,0.110664,0.092240,0.068990,0.076588,0.064833,0.058543,0.049305,0.035943,0.026366,0.031131,0.017331,0.001694,-0.009736,-0.014255,-0.015920,-0.021284,-0.008742,-0.037421,-0.035024,-0.040109,-0.009489,0.007305,0.029458,0.015903,0.003310,-0.040229
1,2000-01-02,0.092635,0.107413,0.108822,0.108591,0.111967,0.111211,0.119137,0.116468,0.120271,0.116661,0.101254,0.098793,0.109843,0.072183,0.070388,0.074738,0.064502,0.077777,0.083013,0.083013,0.104342,0.075977,0.085660,0.080385,0.090332,-0.035767,-0.005061,0.017978,0.036510,0.018628,0.008605,-0.016751
2,2000-01-03,0.132933,0.111175,0.102088,0.092935,0.091619,0.087737,0.089550,0.080550,0.045984,0.025300,0.020605,0.034820,0.055351,0.061460,0.068001,0.070566,0.112957,0.103677,0.108457,0.120620,0.116043,0.125302,0.154267,0.093937,0.108526,-0.081055,0.013123,0.017057,0.031607,0.007759,0.001593,0.187280
3,2000-01-04,0.019185,0.023567,0.046560,0.033526,0.054471,0.060460,0.068659,0.065226,0.065841,0.024099,0.018128,0.013584,0.007942,0.003133,-0.027661,-0.019598,-0.010939,0.000233,-0.005809,-0.002867,0.013325,0.008685,0.025226,0.049480,0.052285,0.022559,0.019400,0.022753,0.043048,0.022607,0.016058,-0.003890
4,2000-01-05,0.004970,-0.007078,-0.009864,-0.000072,0.003017,0.014748,0.014504,0.016163,0.008998,0.035545,0.028835,0.036682,0.041242,0.036976,0.053052,0.062109,0.054398,0.057881,0.079518,0.072934,0.087000,0.090995,0.122150,0.112607,0.164821,-0.001434,0.011741,0.019596,0.040089,0.014866,0.023866,0.046271


In [45]:
#N = 25  # of firms in an economy, here # of portfolios, here len(data)
T = len(df)  # length of periods # here 1136#df.columns.array
#  data
data = df.loc[:,'p1':'p25']
#data.shape#(T,N)
#x,y=data.shape
#N=y#  # of firms in an economy, here # of portfolios, here len(data)
#T = data.shape[0]
N = data.shape[1]

In [46]:
# simulate factor loadings (ie, coefficients)
b_mkt = np.zeros((1,N)) # true loadings for market factor
b_smb = np.zeros((1,N)) # true loadings for size factor
b_hml = np.zeros((1,N)) # true loadings for value factor
b_cma = np.zeros((1,N)) # true loadings for value factor
b_rmw = np.zeros((1,N)) # true loadings for value factor

In [47]:
#  factor returns
mkt = df['Mkt_rf'] # market factor
smb = df['SMB'] # size factor
hml = df['HML'] # value factor
cma = df['CMA'] # market factor
rmw = df['RMW'] # size factor


In [48]:
#data.iloc[:,1]
fctrs = df[["Mkt_rf","SMB","HML","CMA","RMW"]]
fctrs.head()


,Mkt_rf,SMB,HML,CMA,RMW
0,-0.040109,-0.009489,0.007305,0.015903,0.003310
1,-0.035767,-0.005061,0.017978,0.018628,0.008605
2,-0.081055,0.013123,0.017057,0.007759,0.001593
3,0.022559,0.019400,0.022753,0.022607,0.016058
4,-0.001434,0.011741,0.019596,0.014866,0.023866


In [12]:
no_factors=fctrs.shape[1]

In [13]:
# Step 1: time series regression firm by firm
#fctrs = np.c_[mkt, smb, hml]
# of factors
b = np.zeros(no_factors)
for i in range(N): # firm-by-firm time series regression
  model = LinearRegression()
  model.fit(fctrs,data.iloc[:,i])
  b = np.c_[b,model.coef_]

b  = np.transpose(b[:,1:])
b.shape
#b should be 0.1, 0.2

(25, 5)

In [14]:
#b is bhat
#b
#b.shape# (N,#factors)
df_bhat = pd.DataFrame(b)
df_bhat.columns = ['b_01','b_02','b_03','b_04','b_05']# change according factos
#print(df_bhat.head())

from google.colab import  drive
df_bhat.to_csv('/gdrive/My Drive/data/bhat_ayesha_5factors.csv')

In [16]:
# Step 2: cross sectional regression time-by-time
f = np.zeros(no_factors)
for i in range(T): # time-by-time cross sectional regression
  model = LinearRegression()
  model.fit(b,data.iloc[i,:])
  f = np.c_[f,model.coef_]

f = np.transpose(f[:,1:])
df_gamma = pd.DataFrame(f)
df_gamma.columns = ['mkt','smb','hml','cma','rmw']
print(df_gamma.describe())
print('Median estimate:', np.median(f,axis=0))
print('Standard errors (without Shanken correction):', np.std(f,axis=0)/T**.5)

              mkt         smb         hml         cma         rmw
count  240.000000  240.000000  240.000000  240.000000  240.000000
mean     0.013265    0.000031    0.000506   -0.003498    0.001550
std      0.387671    0.076571    0.151012    0.178531    0.154394
min     -1.574285   -0.359138   -0.541379   -0.522669   -0.472570
25%     -0.272917   -0.045779   -0.090206   -0.111703   -0.093217
50%      0.033865    0.006228   -0.006022   -0.017976   -0.011503
75%      0.293134    0.047431    0.097338    0.108795    0.092404
max      0.945527    0.178969    0.679910    0.687615    0.547832
Median estimate: [ 0.03386484  0.0062276  -0.00602163 -0.01797589 -0.01150257]
Standard errors (without Shanken correction): [0.02497186 0.00493234 0.00972746 0.0115001  0.0099453 ]


In [ ]:
mu=np.median(f,axis=0)
sigma=(np.std(f,axis=0))/T**.5
tstat=mu/sigma
tstat
#tstat<1.96

array([-0.31781258,  2.83061892,  1.1159732 ])

In [18]:
f.shape # (T,#factors)# f or df I assume is gamma hat
df_gamma.to_csv('/gdrive/My Drive/data/ayesha_gamma_5factors.csv')

In [19]:
df.head()#f and df are same, gamma hat

,Date,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p20,p21,p22,p23,p24,p25,Mkt_rf,SMB,HML,WML,CMA,RMW,Reg
0,2000-01-01,0.178104,0.149602,0.142498,0.121144,0.110616,0.117735,0.110664,0.092240,0.068990,0.076588,0.064833,0.058543,0.049305,0.035943,0.026366,0.031131,0.017331,0.001694,-0.009736,-0.014255,-0.015920,-0.021284,-0.008742,-0.037421,-0.035024,-0.040109,-0.009489,0.007305,0.029458,0.015903,0.003310,-0.040229
1,2000-01-02,0.092635,0.107413,0.108822,0.108591,0.111967,0.111211,0.119137,0.116468,0.120271,0.116661,0.101254,0.098793,0.109843,0.072183,0.070388,0.074738,0.064502,0.077777,0.083013,0.083013,0.104342,0.075977,0.085660,0.080385,0.090332,-0.035767,-0.005061,0.017978,0.036510,0.018628,0.008605,-0.016751
2,2000-01-03,0.132933,0.111175,0.102088,0.092935,0.091619,0.087737,0.089550,0.080550,0.045984,0.025300,0.020605,0.034820,0.055351,0.061460,0.068001,0.070566,0.112957,0.103677,0.108457,0.120620,0.116043,0.125302,0.154267,0.093937,0.108526,-0.081055,0.013123,0.017057,0.031607,0.007759,0.001593,0.187280
3,2000-01-04,0.019185,0.023567,0.046560,0.033526,0.054471,0.060460,0.068659,0.065226,0.065841,0.024099,0.018128,0.013584,0.007942,0.003133,-0.027661,-0.019598,-0.010939,0.000233,-0.005809,-0.002867,0.013325,0.008685,0.025226,0.049480,0.052285,0.022559,0.019400,0.022753,0.043048,0.022607,0.016058,-0.003890
4,2000-01-05,0.004970,-0.007078,-0.009864,-0.000072,0.003017,0.014748,0.014504,0.016163,0.008998,0.035545,0.028835,0.036682,0.041242,0.036976,0.053052,0.062109,0.054398,0.057881,0.079518,0.072934,0.087000,0.090995,0.122150,0.112607,0.164821,-0.001434,0.011741,0.019596,0.040089,0.014866,0.023866,0.046271


In [ ]:
# regrssion using hac newey west method
average(data[:,])

In [ ]:
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import statsmodels.stats.api as sms

In [21]:
data_a=data.to_numpy()
fctrs_a=fctrs.to_numpy()

In [ ]:
#data_a
#fctrs_a

In [22]:
# Compute standard error with bootstrapping (Shanken correction unnecessary)
# One simple bootstrapping 

indices = np.random.randint(low=0,high=T, size=T)
new_data = data_a[indices,:]
new_fctrs = fctrs_a[indices,:]
xx = indices[0]
print(xx)
print(new_data[0,:])
print(data_a[xx,:])


127
[-0.14076684 -0.1100659  -0.10210769 -0.0910169  -0.08560311 -0.0872055
 -0.07770054 -0.07265258 -0.06352705 -0.05991406 -0.06513385 -0.07448504
 -0.07564215 -0.07763806 -0.07317365 -0.06750722 -0.0557904  -0.05165262
 -0.05443453 -0.05039197 -0.04863774 -0.03487689 -0.04349137 -0.02986211
 -0.02525978]
[-0.14076684 -0.1100659  -0.10210769 -0.0910169  -0.08560311 -0.0872055
 -0.07770054 -0.07265258 -0.06352705 -0.05991406 -0.06513385 -0.07448504
 -0.07564215 -0.07763806 -0.07317365 -0.06750722 -0.0557904  -0.05165262
 -0.05443453 -0.05039197 -0.04863774 -0.03487689 -0.04349137 -0.02986211
 -0.02525978]


In [23]:
def efhats(data,fctrs): # function to compute average estimated factor return
  T = data.shape[0]
  N = data.shape[1]
  # Step 1: Time series regression
  x = np.c_[np.ones(T),fctrs]
  M = np.linalg.inv(x.T @ x) @ x.T
  bhat = (M @ data)[1:,:].T

  # Step 2: cross sectional regression
  # Include other control variables here (eg, firm characteristics)
  x = np.c_[np.ones(N),bhat]
  M = np.linalg.inv(x.T @ x) @ x.T
  fhat = (M @ data.T)[1:,:].T
  df = pd.DataFrame(fhat)
  df.columns = ['mkt','smb','hml','cma','rmw']
  E_fhat = np.average(fhat,axis=0)
  return E_fhat 


In [32]:
E_fhat = efhats(data_a,fctrs_a)
simn = 10**3 # of bootstrapping
t = time.time() # bootstrapping start time 
for i in range(simn): 
  indices = np.random.randint(low=0,high=T,size=T) # resample by rows
  new_data = data_a[indices,:] # resampled stock return data
  new_fctrs = fctrs_a[indices,:] # resampled factor return data
  E_fhat = np.c_[E_fhat,efhats(new_data,new_fctrs)]
  # to check bootstrapping status
  if i % 100 == 0: print(i)

print('Simulation duration:', time.time()-t, 'seconds.')
df = pd.DataFrame(E_fhat.T)
df.columns = ['mkt','smb','hml','cma','rmw']
print(df.describe())

0
100
200
300
400
500
600
700
800
900
Simulation duration: 0.982919454574585 seconds.
               mkt          smb          hml          cma          rmw
count  1001.000000  1001.000000  1001.000000  1001.000000  1001.000000
mean      0.002886     0.000790    -0.000519     0.001193     0.003114
std       0.024340     0.005352     0.009849     0.010343     0.008764
min      -0.100461    -0.017788    -0.036823    -0.033360    -0.034511
25%      -0.013149    -0.002627    -0.006417    -0.005474    -0.002742
50%       0.003716     0.000864    -0.000845     0.001228     0.002980
75%       0.018843     0.004152     0.005662     0.008013     0.008509
max       0.083678     0.021657     0.039364     0.031835     0.033042


In [25]:
print('FM regression esimate of expeced factor returns: \n', np.average(E_fhat.T,axis=0))
print('FM regression esimate of standard errors: \n', np.std(E_fhat.T,axis=0))

FM regression esimate of expeced factor returns: 
 [ 0.00380192  0.00076324 -0.00048812  0.0008246   0.00315399]
FM regression esimate of standard errors: 
 [0.02373435 0.00550903 0.00946378 0.00988702 0.00824491]


https://quant.stackexchange.com/questions/35781/how-to-interpret-the-grs-f-test-values
#grs test
https://stackoverflow.com/questions/24074481/fama-macbeth-regression-in-python-pandas-or-statsmodels

In [26]:
pip install rpy2

In [49]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [56]:
%%R -i df -w 5 -h 5 --units in -r 200
# import df from global environment
# make default figure size 5 by 5 inches with 200 dpi resolution

install.packages("GRS.test", repos='http://cran.us.r-project.org', quiet=TRUE)
library(GRS.test)


In [55]:
%%R -i data,fctrs

GRS.MLtest(data_a, fctrs_a)

NULL


In [67]:
%%R -o model -o coef
model<-GRS.MLtest(data, fctrs)
coef<-model$GRS.stat

In [59]:
print(model)

$GRS.stat
          GRS
[1,] 1.107453

$GRS.pval
           GRS
[1,] 0.3360886

$thetas
         [,1]
[1,] 1.052502

$theta
          [,1]
[1,] 0.9314946

$ratio
          [,1]
[1,] 0.8850286




W statistic given in (7) of GRS (1989) <DOI:10.2307/1913625>

GRS.stat=GRS test statistic

GRS.pval=its p-value

theta=maximum Sharpe ratio of the K factor portfolios

thetas=slope of the efficient frontier based on all assets

ratio=theta/thetas, proportion of the potential efficiency

References
Gibbons, Ross, Shanken, 1989. A test of the efficiency of a given portfolio, Econometrica, 57,1121-
1152. <DOI:10.2307/1913625>


https://towardsdatascience.com/guide-to-r-and-python-in-a-single-jupyter-notebook-ff12532eb3ba

https://www.linkedin.com/pulse/interfacing-r-from-python-3-jupyter-notebook-jared-stufft/
how to switch R and python


In [68]:
print(coef)

[[1.10745338]]
